
# MotoTrack Vision — YOLOv8 + ByteTrack (Colab)

Este notebook detecta e **rastreia múltiplas motos** em vídeo quase em tempo real, com **YOLOv8** (Ultralytics) + **ByteTrack** (rastreio integrado).
Você pode:
- **Enviar um vídeo** (recomendado no Colab), **ou** gravar um clipe curto da sua webcam;
- Rodar **apenas na classe “motorcycle”** (COCO) para foco no requisito;
- **Exportar** o resultado em MP4 e visualizar no próprio notebook.

> **Observação:** Colab não oferece acesso direto à webcam em tempo real como no desktop. O notebook inclui uma célula para gravar um clipe curto via webcam (JS) e processá-lo.


In [1]:

#@title 🔧 Instalação das dependências (rodar uma vez)
# Instalação com versões compatíveis
!pip install -q "numpy<2" \
               ultralytics supervision==0.21.0 opencv-python-headless==4.10.0.84 \
               lapx==0.5.9 onnxruntime-gpu -U > /dev/null

# Imports necessários
import torch, cv2, os, sys, json, time, glob, shutil
import numpy as np
from pathlib import Path

print("PyTorch CUDA disponível:", torch.cuda.is_available())

try:
    import ultralytics
    from ultralytics import YOLO
    print("Ultralytics versão:", ultralytics.__version__)
except Exception as e:
    print("Erro ao importar ultralytics:", e)

import supervision as sv
print("Supervision versão:", sv.__version__)



PyTorch CUDA disponível: False
Ultralytics versão: 8.3.203
Supervision versão: 0.21.0


In [2]:

#@title 🔍 Info da GPU (opcional)
!nvidia-smi || echo "Sem GPU disponível no ambiente."


/bin/bash: line 1: nvidia-smi: command not found
Sem GPU disponível no ambiente.


In [3]:

#@title 📦 Carregar modelo YOLOv8
# Escolha o tamanho do modelo: 'n', 's', 'm', 'l', 'x' (quanto maior, mais pesado/preciso)
modelo = "yolov8s.pt"  #@param ["yolov8n.pt", "yolov8s.pt", "yolov8m.pt", "yolov8l.pt", "yolov8x.pt"]
conf_thresh = 0.35  #@param {type:"number"}
iou_thresh = 0.45   #@param {type:"number"}

model = YOLO(modelo)
names = model.names
print("Classes do modelo:", names)
print("Classe 'motorcycle' existe?", "motorcycle" in names.values())
motocycle_class_ids = [k for k,v in names.items() if v == "motorcycle"]
print("ID(s) de 'motorcycle':", motocycle_class_ids)


Classes do modelo: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell 

## 📤 Enviar um vídeo (recomendado)

In [4]:

from google.colab import files
import os, shutil

print("Envie um arquivo de vídeo (mp4, mov, avi, mkv...)")
up = files.upload()

# Pega o primeiro arquivo enviado (qualquer extensão/nome)
uploaded_filename = next(iter(up))
print("Arquivo original:", uploaded_filename)

# Define um nome fixo para usar no restante do notebook
video_path = "video_input.mp4"

# Copia/renomeia para garantir sempre o mesmo nome
shutil.move(uploaded_filename, video_path)

print("Arquivo padronizado como:", video_path)


Envie um arquivo de vídeo (mp4, mov, avi, mkv...)


Saving teste.mp4 to teste.mp4
Arquivo original: teste.mp4
Arquivo padronizado como: video_input.mp4


## 🏍️ Detecção + Rastreamento (ByteTrack integrado)

In [5]:
#@title Rodar rastreamento apenas na classe "motorcycle" (FIX reprodução)
from IPython.display import HTML
import os, glob, shutil, sys, subprocess

assert 'video_path' in globals() and video_path is not None, "Nenhum vídeo foi definido. Envie um vídeo ou grave um clipe de webcam."
output_path = "resultado_motos.mp4"

# IDs da classe 'motorcycle'
motorcycle_ids = [k for k, v in model.names.items() if v == "motorcycle"]
if not motorcycle_ids:
    raise RuntimeError("Classe 'motorcycle' não encontrada no modelo atual.")

# Tracking
results = model.track(
    source=video_path,
    conf=conf_thresh,
    iou=iou_thresh,
    classes=motorcycle_ids,
    tracker="bytetrack.yaml",
    save=True,
    project="runs/moto_track",
    name="exp",
    exist_ok=True,
    persist=True,
    verbose=False
)

# Procurar o arquivo de saída mais recente (qualquer extensão comum de vídeo)
exts = ("*.mp4","*.avi","*.mov","*.mkv")
cands = []
for e in exts:
    cands += glob.glob(os.path.join("runs/moto_track/exp", e))
if not cands:
    raise RuntimeError("Nenhum arquivo de vídeo encontrado em runs/moto_track/exp.")

cands = sorted(cands, key=os.path.getmtime, reverse=True)
latest = cands[0]
latest_ext = os.path.splitext(latest)[1].lower()

def convert_to_mp4(src, dst):
    # Usa ffmpeg (já disponível no Colab) para garantir MP4/H.264 tocável no browser
    cmd = ["ffmpeg","-y","-hide_banner","-loglevel","error","-i", src,
           "-vcodec","libx264","-acodec","aac","-movflags","+faststart", dst]
    subprocess.check_call(cmd)

# Se já for mp4, apenas copie; senão, converta
if latest_ext == ".mp4":
    shutil.copyfile(latest, output_path)
else:
    convert_to_mp4(latest, output_path)

print("Results saved to runs/moto_track/exp")
print("Saída:", output_path)



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to /content/runs/moto_track/exp
Results saved to runs/moto_track/exp
Saída: resultado_motos.mp4


## 📈 Contagem de objetos e estatísticas simples

In [6]:

#@title Reprocessar para obter contagem total de rastros únicos
import cv2, numpy as np
from collections import defaultdict

# Processa novamente (rápido) para extrair IDs dos rastros
# Usamos stream=True para iterar os resultados sem salvar vídeo novamente
track_ids = set()
for r in model.track(
    source=video_path,
    conf=conf_thresh,
    iou=iou_thresh,
    classes=motorcycle_ids,
    tracker="bytetrack.yaml",
    stream=True,
    persist=True,
    verbose=False
):
    if r.boxes.id is not None:
        for tid in r.boxes.id.int().cpu().tolist():
            track_ids.add(tid)

print(f"Total de motos únicas rastreadas: {len(track_ids)}")


Total de motos únicas rastreadas: 15



## 💾 Baixar o resultado
Execute a célula abaixo para baixar o MP4 com as detecções e IDs.


In [7]:

from google.colab import files
files.download("resultado_motos.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
from google.colab import files
uploaded = files.upload()  # selecione o app.py (e requirements.txt se quiser)


Saving app.py to app (1).py


In [9]:
# ▶️ Subir Streamlit no Colab (sem ngrok) + diagnóstico 404
import sys, os, time, importlib, subprocess, socket, textwrap

APP_PATH = "app.py"      # ajuste se o seu arquivo tiver outro nome
PORT = 8501

def ensure(pkg, pip_name=None):
    pip_name = pip_name or pkg
    try:
        importlib.import_module(pkg)
        print(f"[ok] {pkg} já instalado")
    except Exception:
        print(f"[instalando] {pip_name}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pip_name])

ensure("streamlit")
ensure("requests")

# cria um app mínimo se não existir, só para validar a porta
if not os.path.exists(APP_PATH):
    print(f"[warn] {APP_PATH} não encontrado; criando um app mínimo para teste.")
    with open(APP_PATH, "w", encoding="utf-8") as f:
        f.write(textwrap.dedent("""
        import streamlit as st
        st.set_page_config(page_title="Hello Streamlit", layout="wide")
        st.title("✅ Streamlit está rodando")
        st.write("Se você está vendo esta página, a porta e o proxy do Colab funcionaram.")
        """))

# encerra instâncias antigas, se houver
try:
    subprocess.call(["pkill", "-f", "streamlit run"])
except Exception:
    pass

# inicia usando o Python do kernel (evita erro de PATH)
logfile = "/tmp/streamlit.log"
proc = subprocess.Popen([
    sys.executable, "-m", "streamlit", "run", APP_PATH,
    "--server.port", str(PORT),
    "--server.address", "0.0.0.0",          # bind amplo
    "--server.headless", "true",
    "--server.enableCORS", "false",
    "--server.enableXsrfProtection", "false",
    "--browser.gatherUsageStats", "false",
], stdout=open(logfile, "w"), stderr=subprocess.STDOUT)

print("⏳ Iniciando Streamlit… aguardando disponibilidade da porta.")

# aguarda socket abrir e responder HTTP
import requests
def is_ready():
    try:
        r = requests.get(f"http://127.0.0.1:{PORT}/_stcore/health", timeout=0.8)
        return r.status_code == 200
    except Exception:
        return False

deadline = time.time() + 30  # até 30s
ready = False
while time.time() < deadline:
    if is_ready():
        ready = True
        break
    # checa se o processo morreu
    if proc.poll() is not None:
        break
    time.sleep(0.6)

if not ready:
    print("❌ Não subiu a tempo. Logs do Streamlit:")
    try:
        print(open(logfile, "r").read()[-4000:])
    except Exception:
        print("(sem logs)")
    raise SystemExit

print("✅ Streamlit pronto na porta", PORT)

# abrir via proxy do Colab (use UMA das opções abaixo)
from google.colab import output

# 1) nova aba/janela
output.serve_kernel_port_as_window(PORT)

# 2) OU em iframe na própria célula:
# output.serve_kernel_port_as_iframe(PORT)


[ok] streamlit já instalado
[ok] requests já instalado
⏳ Iniciando Streamlit… aguardando disponibilidade da porta.
✅ Streamlit pronto na porta 8501
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>